In [39]:
import psycopg2
import numpy as np
import pandas as pd

In [51]:
user = 'william'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
passwd = ''
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                   password=passwd)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
query = '''
with patients as (
SELECT DISTINCT ON(dataset.subject_id) subject_id, dataset.hadm_id,
dataset.icustay_id, 
dataset.age, 
dataset.mort_icu
FROM mimiciii.datasetsapsii dataset)

SELECT
patients.*, icustays.los, notes.row_id, notes.iserror, TRIM(notes.category) category, notes.description, notes.text,
round(CAST(EXTRACT(EPOCH FROM notes.charttime - icustays.intime) / 60 / 60 as numeric), 0) as icu_time_hr, s.sapsii_prob
FROM patients
JOIN mimiciii.noteevents notes ON patients.hadm_id = notes.hadm_id
JOIN mimiciii.icustays icustays ON patients.hadm_id = icustays.hadm_id AND patients.icustay_id = icustays.icustay_id
JOIN mimiciii.sapsii s ON
patients.subject_id = s.subject_id AND patients.icustay_id = s.icustay_id
WHERE notes.charttime BETWEEN (icustays.intime) AND (icustays.intime + INTERVAL '2' DAY)
AND notes.charttime IS NOT NULL


ORDER BY patients.subject_id, icu_time_hr ASC



'''
notes = pd.read_sql_query(query, con)

In [52]:
notes.head()

,subject_id,hadm_id,icustay_id,age,mort_icu,los,row_id,iserror,category,description,text,icu_time_hr,sapsii_prob
0,3,145834,211552,76.526792,0,6.0646,768818,None,Radiology,CHEST (PORTABLE AP),[**2101-10-20**] 10:23 PM\n CHEST (PORTABLE AP...,3.0,0.918976
1,3,145834,211552,76.526792,0,6.0646,768829,None,Radiology,CHEST (PORTABLE AP),[**2101-10-21**] 1:00 AM\n CHEST (PORTABLE AP)...,6.0,0.918976
2,3,145834,211552,76.526792,0,6.0646,768834,None,Radiology,CHEST (PORTABLE AP),[**2101-10-21**] 6:10 AM\n CHEST (PORTABLE AP)...,11.0,0.918976
3,3,145834,211552,76.526792,0,6.0646,1260684,None,Nursing/other,Report,Micu Progress Nursing Note:\n\nPatient arrived...,12.0,0.918976
4,3,145834,211552,76.526792,0,6.0646,1260685,None,Nursing/other,Report,MICU NSG PROG NOTE: days\nRemains stable on hi...,19.0,0.918976


In [50]:
notes.to_pickle('MIMICIII_sapsii_dataset_medical-notes-w-sapsii.pickle')
con.close()